# Rat RNN

### Load Dependencies

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import tensorflow as tf

from unityagents import UnityEnvironment

%matplotlib inline

### Hyperparameters

In [ ]:
learning_rate = 1e-4
weights_regularization = 0.2
act_regularization = 0.2
hidden_units = 128
train_steps = 10000
ep_length = 500
sim_steps = 1000000
batch_size = 512

tau = 10.0
noise_stddev = 0.5
zero_speed_probability = 0.5

# [rotation.y, velocity.x, velocity.z, angularVelocity.y, velocity.magnitude, heading, velocity]
x_components = [0, 4]

unity_params = {
    'start_area_extents': 8.0,
    'ep_length': ep_length}

model_path = 'rat_model_rnn'
data_path = 'rat_data_large_rnd'

## Generate Data
Prerequisite: Load unity project in the Editor. Build and save executable as "rat.exe" or "rat.app".

### Start Environment

In [ ]:
env_name = "rat" # Name of the Unity environment binary to launch
env = UnityEnvironment(file_name=env_name, worker_id=1)

# Examine environment parameters
print(str(env))

# Set the default brain to work with
default_brain = env.brain_names[0]

### Collect States

In [ ]:
x,z,ep = [], [], []
input_x = []
a = env.reset(config=unity_params)[default_brain]
for i in range(sim_steps - 1):
    
    # check for NANs
    if a.states[0].dtype != np.float64:
        print(i, a.states[0].dtype, a.states[0])
    elif np.any(np.isnan(a.states[0])):
        print(i, "NAN", a.states[0])
        
    # append
    x.append(a.states[0][0])
    z.append(a.states[0][1])
    input_x.append(a.states[0][2:-1])
    ep.append(a.states[0][-1])

    # next simulation step
    a = env.step()[default_brain]

    if i % (sim_steps/10) == 0 and i != 0:
        print(i)

In [ ]:
x = np.array(x)
z = np.array(z)
input_x = np.array(input_x)
input_y = np.stack((x, z), axis=1)

### Save Training Data

In [ ]:
if not os.path.exists(data_path):
    os.makedirs(data_path)

save_data = {"x": input_x, "y": input_y}
pickle.dump(save_data, open(data_path+"/data.p", "wb"))

## Load Training Data

In [ ]:
save_data = pickle.load(open(data_path+"/data.p", "rb"))
input_x = save_data["x"]
input_y = save_data["y"]
x = input_y[:, 0]
z = input_y[:, 1]

In [ ]:
# [rotation.y, velocity.x, velocity.z, angularVelocity.y, velocity.magnitude, heading, velocity]
print(np.shape(input_x))
input_x = input_x[:, x_components] # rotation and velocity
input_x[:,-1] /= np.max(abs(input_x[:,-1]))
print(np.shape(input_x))

In [ ]:
print(np.min(input_x[:,0]), np.max(input_x[:,0]))
print(np.min(input_x[:,1]), np.max(input_x[:,1]))

In [ ]:
print(np.max(np.abs(input_y[:, 0])), np.max(np.abs(input_y[:, 1])))
input_y[:, 0] /= np.max(np.abs(input_y[:, 0]))
input_y[:, 1] /= np.max(np.abs(input_y[:, 1]))
x = input_y[:, 0]
z = input_y[:, 1]

maze_extents = np.max(np.abs(input_y[:, :]))
maze_extents *= 1.125 # plus 12% for borders

print(maze_extents)

### Visualize Rat Trajectory

In [ ]:
def create_colors(length, segments):
    c_list = []
    for i in range(segments):
        color = np.zeros([length//segments, 4])
        color[:, 3] = 0.5
        color[:, 0:3] = np.random.uniform(0, 1, size=[1, 3])
        c_list.append(color)
    c_list = np.reshape(c_list, [length, 4])
    return c_list

In [ ]:
plt.figure(figsize=(15, 15))
plt.scatter(x, z, s=1, c=create_colors(sim_steps, sim_steps//ep_length))
plt.ylim(-maze_extents, maze_extents)
plt.xlim(-maze_extents, maze_extents)

In [ ]:
plt.figure(figsize=(15,10))
for i in range(15):
    pointer = i
    batch_y = input_y[pointer*ep_length:(pointer+1)*ep_length]
    
    plt.subplot(3, 5, 1 + i)
    plt.scatter(batch_y[:,0], batch_y[:, 1], s=1, c=[0,1,0,1])
    plt.ylim(-maze_extents, maze_extents)
    plt.xlim(-maze_extents, maze_extents)
plt.show()

## Rat RNN

### Define Network

In [ ]:
class MyBasicLSTMCell(tf.contrib.rnn.RNNCell):
  def __init__(self, num_units, noise_stddev, forget_bias=1.0,
               activation=None, reuse=None):
    super(MyBasicLSTMCell, self).__init__(_reuse=reuse)
    self._num_units = num_units
    self._noise_stddev = noise_stddev
    self._forget_bias = forget_bias
    self._activation = activation or tf.nn.tanh
    self._linear = None

  @property
  def state_size(self):
    return (tf.contrib.rnn.LSTMStateTuple(self._num_units, self._num_units))

  @property
  def output_size(self):
    return self._num_units

  def call(self, inputs, state):
    sigmoid = tf.sigmoid
    c, h = state

    if self._linear is None:
      self._linear = _Linear([inputs, h], 4 * self._num_units, True)
    # i = input_gate, j = new_input, f = forget_gate, o = output_gate
    i, j, f, o = tf.split(
        value=self._linear([inputs, h]), num_or_size_splits=4, axis=1)

    new_c = (
        c * sigmoid(f + self._forget_bias) + sigmoid(i) * self._activation(j))
    new_h = self._activation(new_c) * sigmoid(o)
    new_h = tf.add(new_h,
                   tf.random_normal(shape=tf.shape(new_h), mean=0.0, stddev=self._noise_stddev, dtype=tf.float32))

    new_state = tf.contrib.rnn.LSTMStateTuple(new_c, new_h)
    return new_h, new_state


from tensorflow.python.ops.rnn_cell_impl import _Linear

class MyCTRNNCell(tf.contrib.rnn.RNNCell):
  def __init__(self, num_units, tau, noise_stddev, activation=None, reuse=None):
    super(MyCTRNNCell, self).__init__(_reuse=reuse)
    self._num_units = num_units
    self._tau = tau
    self._noise_stddev = noise_stddev
    self._activation = activation or tf.nn.tanh
    self._linear = None

  @property
  def state_size(self):
    return (tf.contrib.rnn.LSTMStateTuple(self._num_units, self._num_units))

  @property
  def output_size(self):
    return self._num_units

  def call(self, inputs, state):
    """ Continuous Time RNN:
        dx/dt = -x + W * input + U * act(x) + B + noise
        x' = x + 1/tau * dx/dt
    """
    
    old_x, old_u = state
    new_u = self._activation(old_x)
    
    if self._linear is None:
        self._linear = _Linear([inputs, new_u], self._num_units, True)

    dxdt = self._linear([inputs, new_u]) - old_x
    dxdt = tf.add(dxdt,
                   tf.random_normal(shape=tf.shape(dxdt), mean=0.0, stddev=self._noise_stddev, dtype=tf.float32))
    new_x = old_x + 1.0/self._tau * dxdt
    return new_x, tf.contrib.rnn.LSTMStateTuple(new_x, new_u)

In [ ]:
class RatRNN(object):
    def __init__(self, x_size, y_size, h_size, batch_size, lr):
        self.x = tf.placeholder(shape=[None, x_size], dtype=tf.float32)
        self.train_length = tf.placeholder(shape=[1], dtype=tf.int32)
        #self.batch_size = tf.placeholder(shape=[1], dtype=tf.int32)
        self.noise_stddev = tf.placeholder(shape=[1], dtype=tf.float32)

        self.rnn_in = tf.reshape(self.x, shape=[batch_size, self.train_length[0], x_size])

        cell = MyCTRNNCell(h_size, tau, self.noise_stddev)
        self.state_in = cell.zero_state(batch_size, tf.float32)

        self.rnn, self.rnn_state = tf.nn.dynamic_rnn(inputs=self.rnn_in, cell=cell, dtype=tf.float32,
                                                     initial_state=self.state_in)
        
        self.output = tf.reshape(self.rnn, shape=[-1, h_size])  
        
        self.y_pred = tf.layers.dense(self.output, y_size,
                                      kernel_initializer=tf.zeros_initializer(),
                                      bias_initializer=tf.zeros_initializer(),
                                      activation=None)

        self.y = tf.placeholder(shape=[None, y_size], dtype=tf.float32)
        
        lossL2A = tf.nn.l2_loss(self.output) / float(batch_size * h_size * ep_length)
        lossL2W = tf.add_n([ tf.nn.l2_loss(v) for v in tf.trainable_variables() if 'bias' not in v.name ]) / float(h_size)

        self.weights_loss = lossL2W * weights_regularization
        self.act_loss = lossL2A * act_regularization
        self.regress_loss = tf.reduce_sum(tf.squared_difference(self.y, self.y_pred)) / float(batch_size * ep_length)
        self.loss = self.regress_loss + self.weights_loss + self.act_loss

        self.loss = tf.Print(self.loss, [lossL2A, lossL2W, self.regress_loss])

        optimizer = tf.train.AdamOptimizer(learning_rate=lr)
        self.update = optimizer.minimize(self.loss)

In [ ]:
tf.reset_default_graph()
rat_rnn = RatRNN(len(x_components), 2, hidden_units, batch_size, learning_rate)
sess = tf.InteractiveSession()
saver = tf.train.Saver()

print(tf.trainable_variables())

### Train Network

In [ ]:
losses = []
init = tf.global_variables_initializer()
sess.run(init)
print(ep_length, sim_steps/ep_length)

# center training data
input_Y = np.copy(input_y)
for i in range(sim_steps//ep_length):
    pointer = i
    input_Y[pointer*ep_length:(pointer+1)*ep_length] -= input_Y[pointer*ep_length]

# debug viz
plt.figure(figsize=(5,3))
for i in range(15):
    pointer = i
    batch_y = input_Y[pointer*ep_length:(pointer+1)*ep_length]
    
    plt.subplot(3, 5, 1 + i)
    plt.scatter(batch_y[:,0], batch_y[:, 1], s=1, c=[0,1,0,1])
    plt.ylim(-maze_extents, maze_extents)
    plt.xlim(-maze_extents, maze_extents)
plt.show()

In [ ]:
for i in range(train_steps):
    pointer = np.random.randint(0, sim_steps/ep_length - batch_size)
    batch_x = input_x[pointer*ep_length:(pointer+batch_size)*ep_length]
    batch_y = input_Y[pointer*ep_length:(pointer+batch_size)*ep_length]
    
    batch_x[:,-1] *= np.random.choice(a=[0.0, 1.0], size=(len(batch_x[:,-1])),
                                      p=[zero_speed_probability, 1-zero_speed_probability])
    
    feed_dict = {rat_rnn.x: np.reshape(batch_x, [-1,len(x_components)]), rat_rnn.y: np.reshape(batch_y, [-1, 2]), 
                 rat_rnn.train_length: [ep_length], rat_rnn.noise_stddev: [noise_stddev] }
    loss, _ = sess.run([rat_rnn.loss, rat_rnn.update], feed_dict=feed_dict)
    if i % 100 == 0 and i != 0:
        print("Loss: {} -- {}".format(loss, i//100))
    losses.append(loss)
plt.plot(losses)

In [ ]:
learning_rate *= .5
print(learning_rate)
train_steps = 4000

In [ ]:
plt.plot(losses)

### Save Network Weights

In [ ]:
if not os.path.exists(model_path):
    os.makedirs(model_path)

saver.save(sess, model_path + '/model.cptk')

## Load Network Weights

In [ ]:
ckpt = tf.train.get_checkpoint_state(model_path)
if ckpt == None:
  print('The model {0} could not be found. Make sure you specified the right '
    '--run-path'.format(model_path))
saver.restore(sess, ckpt.model_checkpoint_path)

### Compare Trajectories

In [ ]:
plt.figure(figsize=(15,10))
for i in range(1):
    pointer = i
    batch_x = input_x[pointer*ep_length:(pointer+batch_size)*ep_length]
    batch_y = input_Y[pointer*ep_length:(pointer+batch_size)*ep_length]

    feed_dict = {rat_rnn.x: np.reshape(batch_x, [-1,len(x_components)]), 
                 rat_rnn.train_length: [ep_length], rat_rnn.noise_stddev: [1*noise_stddev]}
    yp = sess.run(rat_rnn.y_pred, feed_dict=feed_dict) + batch_y[0]
    
for i in range(min(15, batch_size)):
    # Real - Green | Predicted - Blue
    plt.subplot(3, 5, 1 + i)
    a = i*ep_length
    b = a+ep_length
    plt.scatter(batch_y[a:b,0], batch_y[a:b, 1], s=1, c=[0,1,0,1])
    plt.scatter(yp[a:b, 0], yp[a:b, 1], s=1, c=[0,0,1,1])
    plt.ylim(-maze_extents, maze_extents)
    plt.xlim(-maze_extents, maze_extents)
plt.show()

### Display neuron activation rates

In [ ]:
resolution = 32
rate = np.zeros([hidden_units, resolution, resolution])
count = np.zeros([resolution, resolution])
for i in range(len(input_x)//(ep_length*batch_size)):
    pointer = i
    batch_x = input_x[pointer*ep_length*batch_size:(pointer+1)*ep_length*batch_size]
    batch_y = input_y[pointer*ep_length*batch_size:(pointer+1)*ep_length*batch_size]
    feed_dict = {rat_rnn.x: np.reshape(batch_x, [-1,len(x_components)]), 
                 rat_rnn.train_length: [ep_length], rat_rnn.noise_stddev: [0*noise_stddev]}
    act = sess.run(rat_rnn.output, feed_dict=feed_dict)
    
    for j in range(ep_length*batch_size):
        x = (batch_y[j][0] + maze_extents)/(maze_extents*2) * resolution
        y = (batch_y[j][1] + maze_extents)/(maze_extents*2) * resolution
        count[int(x), int(y)] += 1
        rate[:, int(x), int(y)] += np.abs(act[j, :])
                      
for x in range(resolution):
    for y in range(resolution):
        if count[x, y] > 0:
            rate[:, x, y] /= count[x, y]

In [ ]:
            
plt.figure(figsize=(15,30))
for h in range(hidden_units):
    plt.subplot(hidden_units//8, 8, 1 + h)
    plt.title('Neuron ' + str(h))
    plt.imshow(rate[h,:,:] / np.max(rate[h,:,:]), interpolation="nearest", cmap="plasma")
    plt.axis('off')
    
plt.show()

## Close TensorFlow Session & Environment

In [ ]:
sess.close()
env.close()

In [ ]:
print(input_Y)
print(input_y)